# Wokflow para generar submit a partir del ensemble de metamodelos
en Futuro = 202109
Silvana Contreras


In [1]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

[1] "2025-12-03 20:52:15 -03"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,647783,34.6,1431157,76.5,1075212,57.5
Vcells,1193466,9.2,8388608,64.0,1876958,14.4


In [2]:
PARAM <- list()
PARAM$experimento <- "compe_03_ensembles"
PARAM$semilla_primigenia <- 999199

In [3]:
# setwd("/content/buckets/b1/exp")
# experimento_folder <- PARAM$experimento
# dir.create(experimento_folder, showWarnings=FALSE)
# setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

# Preprocesamiento

In [4]:
Sys.time()
require( "data.table" )

# leo el dataset VM
#dataset_02 <- fread("~/buckets/b1/datasets/competencia_02_crudo.csv.gz" )
#dataset_03 <- fread("~/buckets/b1/datasets/competencia_03_crudo.csv.gz" )

options(timeout = 200)
dataset_02 <- fread("https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/competencia_02_crudo.csv.gz" )
dataset_03 <- fread("https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/competencia_03_crudo.csv.gz" )



[1] "2025-12-03 20:52:15 -03"

Cargando paquete requerido: data.table



In [5]:
# Uno datasets compe02 y 03

if (setequal(names(dataset_02), names(dataset_03))) {
  
  # ordenamos columnas igual por prolijidad
  setcolorder(dataset_03, names(dataset_02))
  
  dataset_0203 <- rbindlist(list(dataset_02, dataset_03), use.names = TRUE)
  
  cat("\n=== DATASET UNIDO ===\n")
  cat("Filas total: ", nrow(dataset_0203), "\n")
  cat("Rango meses: ", min(dataset_0203$foto_mes), " - ", max(dataset_0203$foto_mes), "\n")
  cat("Conteos por mes:\n")
  print(dataset_0203[, .N, by = foto_mes][order(foto_mes)])
  
} else {
  cat("\nNo se unieron porque las columnas no coinciden. Revisar diferencias listadas arriba.\n")
}


=== DATASET UNIDO ===
Filas total:  4883051 
Rango meses:  201901  -  202109 
Conteos por mes:
    foto_mes      N
       <int>  <int>
 1:   201901 124273
 2:   201902 125401
 3:   201903 125967
 4:   201904 126538
 5:   201905 127202
 6:   201906 128734
 7:   201907 130268
 8:   201908 132143
 9:   201909 133757
10:   201910 136010
11:   201911 137996
12:   201912 139980
13:   202001 143464
14:   202002 146660
15:   202003 148798
16:   202004 149350
17:   202005 150792
18:   202006 153102
19:   202007 154984
20:   202008 156461
21:   202009 157805
22:   202010 159169
23:   202011 160250
24:   202012 160997
25:   202101 161527
26:   202102 162155
27:   202103 163109
28:   202104 163418
29:   202105 163917
30:   202106 164313
31:   202107 164596
32:   202108 164822
33:   202109 165093
    foto_mes      N


In [6]:
dataset <- dataset_0203
dim(dataset)

[1] 4883051     154

In [7]:
rm(dataset_0203, dataset_02, dataset_03)
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,759765,40.6,1431157,76.5,1431157,76.5
Vcells,555656860,4239.4,1481511358,11303.1,1314002266,10025.1


### Generacion de la clase_ternaria

In [8]:
Sys.time()
require( "data.table" )

# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
  "pos" = .I,
  numero_de_cliente,
  periodo0 = as.integer(foto_mes/100)*12 +  foto_mes%%100 )
]

# ordeno
setorder( dsimple, numero_de_cliente, periodo0 )

# calculo topes
periodo_ultimo <- dsimple[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1


# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
  shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente
]

# assign most common class values = "CONTINUA"
dsimple[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dsimple[ periodo0 < periodo_ultimo &
  ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
  clase_ternaria := "BAJA+1"
]

# calculo BAJA+2
dsimple[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
  & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
  clase_ternaria := "BAJA+2"
]

# pego el resultado en el dataset original y grabo
setorder( dsimple, pos )
dataset[, clase_ternaria := dsimple$clase_ternaria ]

rm(dsimple)
gc()
Sys.time()

[1] "2025-12-03 20:53:48 -03"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,765330,40.9,1431157,76.5,1431157,76.5
Vcells,560561567,4276.8,1481511358,11303.1,1314002266,10025.1


[1] "2025-12-03 20:53:49 -03"

In [9]:
setorder( dataset, foto_mes, clase_ternaria, numero_de_cliente)
dataset[, .N, list(foto_mes, clase_ternaria)]

foto_mes,clase_ternaria,N
<int>,<chr>,<int>
201901,BAJA+1,645
201901,BAJA+2,729
201901,CONTINUA,122899
201902,BAJA+1,733
201902,BAJA+2,707
201902,CONTINUA,123961
201903,BAJA+1,708
201903,BAJA+2,751
201903,CONTINUA,124508


# Predicciones externas

Se levantan las predicciones originadas a partir de los modelos:

-wfz3316 (ensemble de 30 semillas)
-wfz3317 (ensemble de 30 semillas)
-wfz3336 (ensemble de 50 semillas)
-wfz3336_seed (ensemble de 50 semillas)
-wfz3339 (ensemble de 50 semillas)
-wfz3339_seed (ensemble de 100 semillas)

Se ensamblan esas predicciones con promedio simple.

In [11]:
PARAM$false_future <- c(202109) 
d_false_future <- dataset[foto_mes %in% PARAM$false_future]

#PREDICCIONES 202109
urls_false_future <- c(
"https://storage.googleapis.com/silvanacontreras76_bukito3/exp/wfzR3316/prediccion_future_real_wfzR3316.txt"
,
"https://storage.googleapis.com/silvanafirefox1976_bukito3/exp/wfzR3317/prediccion_future_real_wfzR3317.txt"
,
#"https://storage.googleapis.com/silexactasedge_bukito3/exp/wfzR3335/prediccion_future_real_wfzR3335.txt",
#"https://storage.googleapis.com/silvanacloudbrave_bukito3/exp/wfzR3335_seed/prediccion_future_real_wfzR3335_seed.txt"
#,
"https://storage.googleapis.com/silexactasedge_bukito3/exp/wfzR3336/prediccion_future_real_wfzR3336.txt",
"https://storage.googleapis.com/silvanafirefox1976_bukito3/exp/wfzR3336_seed/prediccion_future_real_wfzR3336_seed.txt"
,
"https://storage.googleapis.com/silexactasedge_bukito3/exp/wfzR3339/prediccion_future_real_wfzR3339.txt",
"https://storage.googleapis.com/silvanacloudbrave_bukito3/exp/wfzR3339_seed/prediccion_future_real_wfzR3339_seed.txt"
)

tb_prediccion_false_future_prom <- fread(urls_false_future[1], sep = "\t")
setorder(tb_prediccion_false_future_prom, numero_de_cliente, foto_mes)

vpred_acum_false <- tb_prediccion_false_future_prom$prob
qacumulados_false <- 1L

if (length(urls_false_future) > 1L) {
  for (url_i in urls_false_future[-1]) {
    dt <- fread(url_i, sep = "\t")
    setorder(dt, numero_de_cliente, foto_mes)

    if (!all(dt$numero_de_cliente == tb_prediccion_false_future_prom$numero_de_cliente) ||
        !all(dt$foto_mes == tb_prediccion_false_future_prom$foto_mes)) {
      stop(sprintf("Las llaves no coinciden en %s", url_i))
    }

    vpred_acum_false <- vpred_acum_false + dt$prob
    qacumulados_false <- qacumulados_false + 1L

    cat(" -> Leída y sumada:", url_i, "\n")
  }
}
vpred_acum_false <- vpred_acum_false / qacumulados_false

tb_prediccion_false_future_prom[, prob := vpred_acum_false]

fwrite(
  tb_prediccion_false_future_prom[, .(numero_de_cliente, foto_mes, prob)],
  file = paste0("prediccion_ext_future_ensemble_", PARAM$experimento, ".txt"),
  sep = "\t"
)

cat("\nPredicción promedio FINAL: prediccion_ext_future_ensemble_", PARAM$experimento, ".txt\n")


 -> Leída y sumada: https://storage.googleapis.com/silvanafirefox1976_bukito3/exp/wfzR3317/prediccion_future_real_wfzR3317.txt 
 -> Leída y sumada: https://storage.googleapis.com/silexactasedge_bukito3/exp/wfzR3336/prediccion_future_real_wfzR3336.txt 
 -> Leída y sumada: https://storage.googleapis.com/silvanafirefox1976_bukito3/exp/wfzR3336_seed/prediccion_future_real_wfzR3336_seed.txt 
 -> Leída y sumada: https://storage.googleapis.com/silexactasedge_bukito3/exp/wfzR3339/prediccion_future_real_wfzR3339.txt 
 -> Leída y sumada: https://storage.googleapis.com/silvanacloudbrave_bukito3/exp/wfzR3339_seed/prediccion_future_real_wfzR3339_seed.txt 

Predicción promedio FINAL: prediccion_ext_future_ensemble_ compe_03_ensembles .txt


### submit at 11000 envíos
Se genera el submit en formato kaggle para la predicción del ensemble de los metamodelos.
Se elige el corte en los 11 mil envíos basado en experimentos de testeo.

In [13]:
# kaggle 

dir.create("kaggle_real", showWarnings = FALSE)

PARAM$cortes <- 11000

tb_ord_test <- copy(tb_prediccion_false_future_prom)
setorder(tb_ord_test, -prob)

for (envios in PARAM$cortes) {
  tb_ord_test[, Predicted := 0L]                          
  tb_ord_test[1:min(envios, .N), Predicted := 1L]        

  archivo_kaggle_real <- paste0("./kaggle_real/3316_3317_3336_3339", PARAM$experimento, "_", envios, ".csv")

  # grabo el archivo
  fwrite(tb_ord_test[, .(numero_de_cliente, Predicted)],
        file = archivo_kaggle_real,
        sep = ",",
        col.names = TRUE)

  cat("Generado:", basename(archivo_kaggle_real), "\n")
}

Generado: 3316_3317_3336_3339compe_03_ensembles_11000.csv 


In [14]:
# genero el formato requerido para la entrega

archivo_kaggle_format <- sub("\\.csv$", "_format\\.csv", archivo_kaggle_real) 

# CARGAR EL ARCHIVO GENERADO
tb_kaggle <- fread(archivo_kaggle_real)

# FILTRAR los registros donde Predicted es 1
tb_format <- tb_kaggle[Predicted == 1]

# IMPLEMENTAR EL CONTROL de 11,000 filas
filas_finales <- nrow(tb_format)
meta_filas <- 11000

if (filas_finales == meta_filas) {
  message(paste(" Control exitoso. Se encontraron", filas_finales, "filas con Predicted = 1."))
  
  fwrite(
    # solo la columna 'numero_de_cliente'
    tb_format[, .(numero_de_cliente)], 
    file = archivo_kaggle_format,
    sep = ",",
    # col.names = FALSE: ELIMINA el encabezado (primera fila)
    col.names = FALSE, 
    quote = FALSE 
  )
  cat("Generado formato:", basename(archivo_kaggle_format), "\n")
  
} else {
  stop(paste(" ERROR: El archivo final debe tener", meta_filas, "filas. Se encontraron", filas_finales, "filas. Revisar la predicción."))
}

 Control exitoso. Se encontraron 11000 filas con Predicted = 1.



Generado formato: 3316_3317_3336_3339compe_03_ensembles_11000_format.csv 
